# Segment raw images to get gray matter before normalising.
This is to prevent errors in normalisation down the line (e.g. enlargement of ventricles: GM WM growth or shrinkage?)

In [1]:
# import everything you need
from nipype import Node, Workflow
import nipype.interfaces.spm as spm
import numpy as np
import pandas as pd
import os, re, random

200210-13:32:57,853 nipype.utils WARNING:
	 Could not check for version updates: 
Connection to server could not be made


In [2]:
# define useful functions

# get subject directories from parent directory
def get_subdirs(dir):
    subdirs=[]
    for i in os.listdir(dir):
        # verify subject number in directory and exclude other dirs
        match=re.match(r'(.*\d{3}$)', i)
        if os.path.isdir(os.path.join(dir, i)) and match:
            subdirs.append(os.path.join(dir, match.group(0)))
    return sorted(subdirs)

In [3]:
subdirs=get_subdirs("/VBM/")
print(subdirs)

['/VBM/FED006', '/VBM/FED007', '/VBM/FED008', '/VBM/FED009', '/VBM/FED010', '/VBM/FED011', '/VBM/FED012', '/VBM/FED013', '/VBM/FED014', '/VBM/FED015', '/VBM/FED016', '/VBM/FED017', '/VBM/FED018', '/VBM/FED019', '/VBM/FED020', '/VBM/FED021', '/VBM/FED022', '/VBM/FED023', '/VBM/FED024', '/VBM/FED025', '/VBM/FED026', '/VBM/FED027', '/VBM/FED028', '/VBM/FED029', '/VBM/FED030', '/VBM/FED031', '/VBM/FED032', '/VBM/FED033', '/VBM/FED034', '/VBM/FED035', '/VBM/FED036', '/VBM/FED037', '/VBM/FED038', '/VBM/FED039', '/VBM/FED040', '/VBM/FED041', '/VBM/FED042', '/VBM/FED043', '/VBM/FED044', '/VBM/FED045', '/VBM/FED046', '/VBM/FED047', '/VBM/FED048', '/VBM/FED049', '/VBM/FED050', '/VBM/FED051', '/VBM/FED052', '/VBM/FED053', '/VBM/FED054', '/VBM/FED055', '/VBM/FED056', '/VBM/FED057', '/VBM/FED058', '/VBM/FED059', '/VBM/FED060', '/VBM/FED061', '/VBM/FED062', '/VBM/FED063', '/VBM/FED064', '/VBM/FED065', '/VBM/FED066', '/VBM/FED067', '/VBM/FED068']


In [4]:
# make dict with all T1-images as pathlike objects
images_T1={}
for i in subdirs:
    for j in os.listdir(i):
        match=re.match(r'(.*T1_MPRAGE.*\d{14,}.*.nii)', j)
        if match:
            images_T1[i]=os.path.join(i, match.group(0))

print(images_T1)

{'/VBM/FED006': '/VBM/FED006/CON_T1_MPRAGE_SAG_ISO_0_9_0005_20141106153429.nii', '/VBM/FED007': '/VBM/FED007/CON_T1_MPRAGE_SAG_ISO_0_9_0005_20141107104226.nii', '/VBM/FED008': '/VBM/FED008/CON_T1_MPRAGE_SAG_ISO_0_9_0005_20141112155009.nii', '/VBM/FED009': '/VBM/FED009/CON_T1_MPRAGE_SAG_ISO_0_9_0005_20141117154954.nii', '/VBM/FED010': '/VBM/FED010/CON_T1_MPRAGE_SAG_ISO_0_9_0005_20141119091904.nii', '/VBM/FED011': '/VBM/FED011/DEP_T1_MPRAGE_SAG_ISO_0_9_0005_20141119161708.nii', '/VBM/FED012': '/VBM/FED012/CON_T1_MPRAGE_SAG_ISO_0_9_0005_20141124150148.nii', '/VBM/FED013': '/VBM/FED013/CON_T1_MPRAGE_SAG_ISO_0_9_0005_20141125093955.nii', '/VBM/FED014': '/VBM/FED014/DEP_T1_MPRAGE_SAG_ISO_0_9_0005_20141125153216.nii', '/VBM/FED015': '/VBM/FED015/CON_T1_MPRAGE_SAG_ISO_0_9_0005_20141202145526.nii', '/VBM/FED016': '/VBM/FED016/CON_T1_MPRAGE_SAG_ISO_0_9_0005_20141202164821.nii', '/VBM/FED017': '/VBM/FED017/CON_T1_MPRAGE_SAG_ISO_0_9_0005_20141204151043.nii', '/VBM/FED018': '/VBM/FED018/DEP_T1_MPRA

In [5]:
# use NewSegment from spm to segment unimportant anatomical landmarks and perform "brain-extraction" (TEST FED006)
#for i in images.keys(): use this when ready to segment full subject list

# classify tissues from tpm.nii
# need to perform segmentation with full tissue information, or brain will be classified as "soft tissue" etc. -> see examples in "/VBM/FED006/"
tissue1 = (('/opt/spm12-r7219/spm12_mcr/spm12/tpm/TPM.nii', 1), 2, (True,True), (False, False)) # GM
tissue2 = (('/opt/spm12-r7219/spm12_mcr/spm12/tpm/TPM.nii', 2), 2, (True,True), (False, False)) # WM
tissue3 = (('/opt/spm12-r7219/spm12_mcr/spm12/tpm/TPM.nii', 3), 2, (True,True), (False, False)) # CSF
tissue4 = (('/opt/spm12-r7219/spm12_mcr/spm12/tpm/TPM.nii', 4), 2, (False,False), (False, False)) # bone
tissue5 = (('/opt/spm12-r7219/spm12_mcr/spm12/tpm/TPM.nii', 5), 2, (False,False), (False, False)) # soft tissue
tissue6 = (('/opt/spm12-r7219/spm12_mcr/spm12/tpm/TPM.nii', 6), 2, (False,False), (False, False)) # air and background

# create SKULLSTRIP segment nodes for each subject
for k,v in images_T1.items():
    segm_skullstrip_T1=Node(spm.preprocess.NewSegment(), name='DARTEL_newsegment')
    segm_skullstrip_T1.inputs.tissues=[tissue1, tissue2, tissue3, tissue4, tissue5, tissue6]
    # do not generate SPM8 and higher compatible jobs
    segm_skullstrip_T1.inputs.use_v8struct=False
    segm_skullstrip_T1.inputs.affine_regularization='none'
    # write forward and inverse deformation fields
    #segm_skullstrip_T1.inputs.write_deformation_fields=[True, True]
    # establish input files from subject
    segm_skullstrip_T1.inputs.channel_files=v
    # establish basedir for output of subject node
    segm_skullstrip_T1.base_dir=k
    results=segm_skullstrip_T1.run()
    print(results.outputs)

200124-15:56:50,878 nipype.workflow INFO:
	 [Node] Setting-up "DARTEL_newsegment" in "/VBM/FED006/DARTEL_newsegment".
200124-15:56:50,885 nipype.workflow INFO:
	 [Node] Running "DARTEL_newsegment" ("nipype.interfaces.spm.preprocess.NewSegment")
200124-16:00:40,560 nipype.workflow INFO:
	 [Node] Finished "DARTEL_newsegment".

bias_corrected_images = <undefined>
bias_field_images = <undefined>
dartel_input_images = [['/VBM/FED006/DARTEL_newsegment/rc1CONt_T1_MPRAGE_SAG_ISO_0_9_0005_20141106153429.nii'], ['/VBM/FED006/DARTEL_newsegment/rc2CONt_T1_MPRAGE_SAG_ISO_0_9_0005_20141106153429.nii'], ['/VBM/FED006/DARTEL_newsegment/rc3CONt_T1_MPRAGE_SAG_ISO_0_9_0005_20141106153429.nii'], [], [], []]
forward_deformation_field = <undefined>
inverse_deformation_field = <undefined>
modulated_class_images = [[], [], [], [], [], []]
native_class_images = [['/VBM/FED006/DARTEL_newsegment/c1CONt_T1_MPRAGE_SAG_ISO_0_9_0005_20141106153429.nii'], ['/VBM/FED006/DARTEL_newsegment/c2CONt_T1_MPRAGE_SAG_ISO_0_9_0

In [5]:
# get segmentation directory for each subject

# define start of segmentation directory name
targetnode="segm"
taskdirs={}

for i in images_T1.keys():
    for j in subdirs:
        # match keys with subdir list
        if j == i: 
            for k in os.listdir(j):
                # go through content of subdirs and look for "true" directories
                if os.path.isdir(os.path.join(j, k)):
                    # regex functions match=exact match of pattern, search=finds elements that contain pattern
                    # regex matches entries in j with targetnode k
                    match=re.match(r'.*(' + targetnode + ').*', k)
                    # ? This is needed to recognize the attribute group from the regex output
                    if match: 
                        taskdirs[j]=os.path.join(j, match.group(0))

create GM template using SPM DARTEL

In [41]:
# split taskdirs in control and depressed

# define and load from .xls(x) files
sub_control_ID="FED_Subject_Assignments.xlsx"
control_ID=pd.read_excel(sub_control_ID, sheet_name="Sheet2").astype(str)
ID=control_ID['Sub Num FED_XXX'].str.zfill(3).tolist()
status=control_ID['BDI 22 Score'].tolist()


control_taskdir={}
depress_taskdir={}

for i in taskdirs.keys():
    for x,y in zip(ID,status):
        if i.rsplit('/',1)[1] == "FED"+x and y == "0":
            control_taskdir[i]=taskdirs[i]

for i in taskdirs.keys():
    if i not in control_taskdir.keys():
        depress_taskdir[i]=taskdirs[i]

# check dicts
print(control_taskdir.keys())
print(depress_taskdir.keys())
# see, if sizes are about equal
print(len(control_taskdir.keys()), len(depress_taskdir.keys()))

dict_keys(['/VBM/FED006', '/VBM/FED007', '/VBM/FED008', '/VBM/FED009', '/VBM/FED010', '/VBM/FED012', '/VBM/FED013', '/VBM/FED015', '/VBM/FED016', '/VBM/FED017', '/VBM/FED019', '/VBM/FED020', '/VBM/FED021', '/VBM/FED022', '/VBM/FED023', '/VBM/FED024', '/VBM/FED025', '/VBM/FED026', '/VBM/FED049', '/VBM/FED050', '/VBM/FED053', '/VBM/FED054', '/VBM/FED056', '/VBM/FED058', '/VBM/FED060', '/VBM/FED061', '/VBM/FED062', '/VBM/FED063', '/VBM/FED064', '/VBM/FED065', '/VBM/FED067', '/VBM/FED068'])
dict_keys(['/VBM/FED011', '/VBM/FED014', '/VBM/FED018', '/VBM/FED027', '/VBM/FED028', '/VBM/FED029', '/VBM/FED030', '/VBM/FED031', '/VBM/FED032', '/VBM/FED033', '/VBM/FED034', '/VBM/FED035', '/VBM/FED036', '/VBM/FED037', '/VBM/FED038', '/VBM/FED039', '/VBM/FED040', '/VBM/FED041', '/VBM/FED042', '/VBM/FED043', '/VBM/FED044', '/VBM/FED045', '/VBM/FED046', '/VBM/FED047', '/VBM/FED048', '/VBM/FED051', '/VBM/FED052', '/VBM/FED055', '/VBM/FED057', '/VBM/FED059', '/VBM/FED066'])
32 31


In [46]:
# create template dict

# only 31 depressed, so take all into template
template_dirs=depress_taskdir.copy()
# collect random keys (31 from control)
keys=random.sample(list(control_taskdir), 31)
# integrate both dicts for template_dirs
for i in keys:
    template_dirs[i]=control_taskdir[i]

# get rc1 and rc2 files as path-like objects
rc1=[]
rc2=[]
# append files to lists
for k,v in template_dirs.items():
    for i in os.listdir(v):
        # look for rc1(GM)
        match1=re.match(r'(^rc1.*.nii)', i)
        if match1:
            rc1.append(os.path.join(v, match1.group(0)))
        # look for rc2(WM)
        match2=re.match(r'(^rc2.*.nii)', i)
        if match2:
            rc2.append(os.path.join(v, match2.group(0)))

# sort lists
rc1=sorted(rc1)
rc2=sorted(rc2)

# print lists
print(rc1, len(rc1))
print(rc2, len(rc2))

['/VBM/FED006/DARTEL_newsegment/rc1CON_T1_MPRAGE_SAG_ISO_0_9_0005_20141106153429.nii', '/VBM/FED007/DARTEL_newsegment/rc1CON_T1_MPRAGE_SAG_ISO_0_9_0005_20141107104226.nii', '/VBM/FED008/DARTEL_newsegment/rc1CON_T1_MPRAGE_SAG_ISO_0_9_0005_20141112155009.nii', '/VBM/FED009/DARTEL_newsegment/rc1CON_T1_MPRAGE_SAG_ISO_0_9_0005_20141117154954.nii', '/VBM/FED010/DARTEL_newsegment/rc1CON_T1_MPRAGE_SAG_ISO_0_9_0005_20141119091904.nii', '/VBM/FED011/DARTEL_newsegment/rc1DEP_T1_MPRAGE_SAG_ISO_0_9_0005_20141119161708.nii', '/VBM/FED012/DARTEL_newsegment/rc1CON_T1_MPRAGE_SAG_ISO_0_9_0005_20141124150148.nii', '/VBM/FED013/DARTEL_newsegment/rc1CON_T1_MPRAGE_SAG_ISO_0_9_0005_20141125093955.nii', '/VBM/FED014/DARTEL_newsegment/rc1DEP_T1_MPRAGE_SAG_ISO_0_9_0005_20141125153216.nii', '/VBM/FED015/DARTEL_newsegment/rc1CON_T1_MPRAGE_SAG_ISO_0_9_0005_20141202145526.nii', '/VBM/FED016/DARTEL_newsegment/rc1CON_T1_MPRAGE_SAG_ISO_0_9_0005_20141202164821.nii', '/VBM/FED017/DARTEL_newsegment/rc1CON_T1_MPRAGE_SAG_I

In [53]:
# check that lists are both in the same subject order and template generation will not make wrong assumptions ...^^
for x,y in zip(rc1,rc2):
    if x.rsplit('/', 3)[1] ==  y.rsplit('/', 3)[1]: # specify files to everything except the "2" !TODO!
        print("Yes!")
        print(x.rsplit('/', 3)[1] "is the same as" y.rsplit('/', 3)[1])
    else:
        print("Noooooooooooooooooooooooooooooooooo")

Yes!
FED006
Yes!
FED007
Yes!
FED008
Yes!
FED009
Yes!
FED010
Yes!
FED011
Yes!
FED012
Yes!
FED013
Yes!
FED014
Yes!
FED015
Yes!
FED016
Yes!
FED017
Yes!
FED018
Yes!
FED019
Yes!
FED020
Yes!
FED021
Yes!
FED022
Yes!
FED023
Yes!
FED024
Yes!
FED025
Yes!
FED026
Yes!
FED027
Yes!
FED028
Yes!
FED029
Yes!
FED030
Yes!
FED031
Yes!
FED032
Yes!
FED033
Yes!
FED034
Yes!
FED035
Yes!
FED036
Yes!
FED037
Yes!
FED038
Yes!
FED039
Yes!
FED040
Yes!
FED041
Yes!
FED042
Yes!
FED043
Yes!
FED044
Yes!
FED045
Yes!
FED046
Yes!
FED047
Yes!
FED048
Yes!
FED049
Yes!
FED050
Yes!
FED051
Yes!
FED052
Yes!
FED053
Yes!
FED054
Yes!
FED055
Yes!
FED057
Yes!
FED058
Yes!
FED059
Yes!
FED060
Yes!
FED061
Yes!
FED062
Yes!
FED063
Yes!
FED064
Yes!
FED065
Yes!
FED066
Yes!
FED067
Yes!
FED068


In [54]:
# create DARTEL node and template
dartel_template=Node(spm.DARTEL(), name='DARTEL_template')
dartel_template.inputs.image_files=[rc1, rc2]
dartel_template.inputs.template_prefix="grouptemplate"
dartel_template.base_dir="/VBM"
# do not generate SPM8 and higher compatible jobs
dartel_template.inputs.use_v8struct=False
results=dartel_template.run()

200210-14:48:34,361 nipype.workflow INFO:
	 [Node] Setting-up "DARTEL_template" in "/VBM/DARTEL_template".
200210-14:48:34,396 nipype.workflow INFO:
	 [Node] Running "DARTEL_template" ("nipype.interfaces.spm.preprocess.DARTEL")
200210-17:37:15,90 nipype.workflow INFO:
	 [Node] Finished "DARTEL_template".


normalise to MNI space

In [55]:
# get required file lists and check that theyre in the same order

# deformation flowfields from nonlinear registration
flowfields=[]
# GM and WM 
c1=[]
c2=[]

# append files to lists
# flowfields
finaltemplate_dir="/VBM/DARTEL_template/"
for i in os.listdir(finaltemplate_dir):
    # look for u_rc1
    match=re.match(r'(^u_rc1.*.nii)', i)
    if match:
        flowfields.append(os.path.join(finaltemplate_dir, match.group(0)))

# check flowfields against subject T1 images to see 
# which were picked to create the template
flowfields_matched={}
for k,v in images_T1.items():
    for i in flowfields:
        # if cropped u_rc1T1 and T1 overlap via sequence and reg-number,
        # associate FEDXXX as key to flowfields-value
            if i.rsplit('/',1)[1][5:49] == v.rsplit('/',1)[1][0:44]:
                flowfields_matched[k]=i

# bring matches into sorted order for final list
flowfields=[]
for k,v in flowfields_matched.items():
    flowfields.append(v)
    

# GM and WM
# create dict where flowfields_matched and taskdirs(segmentation) overlap
intemplate_dirs={}
for i in flowfields_matched.keys():
    for k,v in taskdirs.items():
        if i == k:
            intemplate_dirs[i]=v

# fill c1 and c2 lists
for k,v in intemplate_dirs.items():
    for i in os.listdir(v):
        # look for c1(GM)
        match1=re.match(r'(^c1.*.nii)', i)
        if match1:
            c1.append(os.path.join(v, match1.group(0)))
        # look for c2(WM)
        match2=re.match(r'(^c2.*.nii)', i)
        if match2:
            c2.append(os.path.join(v, match2.group(0)))

In [58]:
# check flowfields_matched
print(flowfields_matched.items())

dict_items([('/VBM/FED006', '/VBM/DARTEL_template/u_rc1CON_T1_MPRAGE_SAG_ISO_0_9_0005_20141106153429_grouptemplate.nii'), ('/VBM/FED007', '/VBM/DARTEL_template/u_rc1CON_T1_MPRAGE_SAG_ISO_0_9_0005_20141107104226_grouptemplate.nii'), ('/VBM/FED008', '/VBM/DARTEL_template/u_rc1CON_T1_MPRAGE_SAG_ISO_0_9_0005_20141112155009_grouptemplate.nii'), ('/VBM/FED009', '/VBM/DARTEL_template/u_rc1CON_T1_MPRAGE_SAG_ISO_0_9_0005_20141117154954_grouptemplate.nii'), ('/VBM/FED010', '/VBM/DARTEL_template/u_rc1CON_T1_MPRAGE_SAG_ISO_0_9_0005_20141119091904_grouptemplate.nii'), ('/VBM/FED011', '/VBM/DARTEL_template/u_rc1DEP_T1_MPRAGE_SAG_ISO_0_9_0005_20141119161708_grouptemplate.nii'), ('/VBM/FED012', '/VBM/DARTEL_template/u_rc1CON_T1_MPRAGE_SAG_ISO_0_9_0005_20141124150148_grouptemplate.nii'), ('/VBM/FED013', '/VBM/DARTEL_template/u_rc1CON_T1_MPRAGE_SAG_ISO_0_9_0005_20141125093955_grouptemplate.nii'), ('/VBM/FED014', '/VBM/DARTEL_template/u_rc1DEP_T1_MPRAGE_SAG_ISO_0_9_0005_20141125153216_grouptemplate.nii')

In [59]:
# check lists order
print(flowfields[0:3])
print(c1[0:3])
print(c2[0:3])

['/VBM/DARTEL_template/u_rc1CON_T1_MPRAGE_SAG_ISO_0_9_0005_20141106153429_grouptemplate.nii', '/VBM/DARTEL_template/u_rc1CON_T1_MPRAGE_SAG_ISO_0_9_0005_20141107104226_grouptemplate.nii', '/VBM/DARTEL_template/u_rc1CON_T1_MPRAGE_SAG_ISO_0_9_0005_20141112155009_grouptemplate.nii']
['/VBM/FED006/DARTEL_newsegment/c1CON_T1_MPRAGE_SAG_ISO_0_9_0005_20141106153429.nii', '/VBM/FED007/DARTEL_newsegment/c1CON_T1_MPRAGE_SAG_ISO_0_9_0005_20141107104226.nii', '/VBM/FED008/DARTEL_newsegment/c1CON_T1_MPRAGE_SAG_ISO_0_9_0005_20141112155009.nii']
['/VBM/FED006/DARTEL_newsegment/c2CON_T1_MPRAGE_SAG_ISO_0_9_0005_20141106153429.nii', '/VBM/FED007/DARTEL_newsegment/c2CON_T1_MPRAGE_SAG_ISO_0_9_0005_20141107104226.nii', '/VBM/FED008/DARTEL_newsegment/c2CON_T1_MPRAGE_SAG_ISO_0_9_0005_20141112155009.nii']


In [60]:
# create DARTEL node to norm2MNI for Gm and WM
tissues={"c1": c1, "c2": c2}
for k,v in tissues.items():
    dartel_norm2mni=Node(spm.DARTELNorm2MNI(), name='DARTEL_norm2mni_'+str(k))
    dartel_norm2mni.inputs.template_file =str(finaltemplate_dir)+"grouptemplate_6.nii"
    dartel_norm2mni.inputs.flowfield_files = flowfields
    dartel_norm2mni.inputs.apply_to_files = v
    dartel_norm2mni.inputs.modulate = True
    dartel_norm2mni.base_dir="/VBM"
    # think about smoothing kernel below ... ROI sizes?
    #dartel_norm2mni.inputs.fwhm =
    # do not generate SPM8 and higher compatible jobs
    dartel_norm2mni.inputs.use_v8struct=False
    results=dartel_norm2mni.run()

200210-17:51:56,956 nipype.workflow INFO:
	 [Node] Setting-up "DARTEL_norm2mni_c1" in "/VBM/DARTEL_norm2mni_c1".
200210-17:51:56,979 nipype.workflow INFO:
	 [Node] Running "DARTEL_norm2mni_c1" ("nipype.interfaces.spm.preprocess.DARTELNorm2MNI")
200210-17:59:20,37 nipype.workflow INFO:
	 [Node] Finished "DARTEL_norm2mni_c1".
200210-17:59:20,45 nipype.workflow INFO:
	 [Node] Setting-up "DARTEL_norm2mni_c2" in "/VBM/DARTEL_norm2mni_c2".
200210-17:59:20,75 nipype.workflow INFO:
	 [Node] Running "DARTEL_norm2mni_c2" ("nipype.interfaces.spm.preprocess.DARTELNorm2MNI")
200210-18:06:39,742 nipype.workflow INFO:
	 [Node] Finished "DARTEL_norm2mni_c2".


Testing Zone:

In [36]:
# check lists order (function^^)
regex=re.compile('\d{14}')
for i in list(flowfields,c1,c2):
    nums=list(filter(regex.match, i))
    

<built-in method groupdict of re.Match object at 0x7fccfc286f30>
<built-in method groupdict of re.Match object at 0x7fccfc128990>
<built-in method groupdict of re.Match object at 0x7fccfc128a80>
<built-in method groupdict of re.Match object at 0x7fccfc1287b0>
<built-in method groupdict of re.Match object at 0x7fccfc0d7c60>
<built-in method groupdict of re.Match object at 0x7fccfc0d7e40>
<built-in method groupdict of re.Match object at 0x7fccfc0d76c0>
<built-in method groupdict of re.Match object at 0x7fccfc0d7be8>
<built-in method groupdict of re.Match object at 0x7fccfc0d7d50>
<built-in method groupdict of re.Match object at 0x7fccfbdc4648>
<built-in method groupdict of re.Match object at 0x7fccfbdc4558>
<built-in method groupdict of re.Match object at 0x7fccfbdc46c0>
<built-in method groupdict of re.Match object at 0x7fccfbdc43f0>
<built-in method groupdict of re.Match object at 0x7fccfbdc4120>
<built-in method groupdict of re.Match object at 0x7fccfbdc40a8>
<built-in method groupdic

In [12]:
# crop down images for testing (only FED006,07,08!!!)
images2=dict()
for i in sorted(images.keys())[0:3]:
    images2[i]=images[i]
    
print(images2.items())

dict_items([('FED006', '/VBM/FED006/T1_MPRAGE_SAG_ISO_0_9_0005_t1_mprage_sag_iso_0.9_20141106153429_5.nii'), ('FED007', '/VBM/FED007/T1_MPRAGE_SAG_ISO_0_9_0005_t1_mprage_sag_iso_0.9_20141107104226_5.nii'), ('FED008', '/VBM/FED008/T1_MPRAGE_SAG_ISO_0_9_0005_t1_mprage_sag_iso_0.9_20141112155009_5.nii')])


In [69]:
# perform operations on images using fslmaths
from nipype.interfaces.fsl import ImageMaths


In [70]:
%%bash
# check if operator -mul is even appropriate in this case
fslmaths -h


Usage: fslmaths [-dt <datatype>] <first_input> [operations and inputs] <output> [-odt <datatype>]

Datatype information:
 -dt sets the datatype used internally for calculations (default float for all except double images)
 -odt sets the output datatype ( default is float )
 Possible datatypes are: char short int float double input
 "input" will set the datatype to that of the original image

Binary operations:
  (some inputs can be either an image or a number)
 -add   : add following input to current image
 -sub   : subtract following input from current image
 -mul   : multiply current image by following input
 -div   : divide current image by following input
 -rem   : modulus remainder - divide current image by following input and take remainder
 -mas   : use (following image>0) to mask current image
 -thr   : use following number to threshold current image (zero anything below the number)
 -thrp  : use following percentage (0-100) of ROBUST RANGE to threshold current image (zero any